In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os,time,pickle
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from tqdm import tqdm

from joblib import dump, load
from utils import data_process, visualize
from utils.utils import train, test, summary, setup_seed
from utils.MCRNN import MCRNN
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "6"

In [2]:
data = np.load("./data_4/data_train.npy", allow_pickle=True)
x_train = data.item().get("x_train")
y_train = data.item().get("y_train")
data = np.load("./data_4/data_test.npy", allow_pickle=True)
x_test = data.item().get("x_test")
y_test = data.item().get("y_test")
x_train=x_train.transpose((0,2,1))
x_test=x_test.transpose((0,2,1))

x_train = torch.from_numpy(x_train).float()
y_train = torch.from_numpy(y_train).long()

x_test = torch.from_numpy(x_test).float()
y_test = torch.from_numpy(y_test).long()
traindata = TensorDataset(x_train, y_train)
testdata = TensorDataset(x_test, y_test)
train_loader = DataLoader(traindata, batch_size=256, shuffle=True)
test_loader = DataLoader(testdata, batch_size=256, shuffle=True)

In [8]:

class Conv(nn.Module):
    def __init__(self, ):
        super(Conv, self).__init__()
        self.conv_1 = nn.Conv1d(in_channels=1,  out_channels=32, kernel_size=3, stride=3)
        self.conv_2 = nn.Conv1d(in_channels=32, out_channels=32, kernel_size=3, stride=3)
        self.conv_3 = nn.Conv1d(in_channels=32, out_channels=32, kernel_size=3)
        self.MaxPool = nn.MaxPool1d(2)
        self.Flatten = torch.nn.Flatten()

    def forward(self, x):
        out = F.relu(self.conv_1(x))
        out = F.relu(self.conv_2(out))
        out = F.relu(self.conv_3(out))
        out = self.MaxPool(out)
        out = self.Flatten(out)
        return out


class MCRNN(nn.Module):
    def __init__(self, ):
        super(MCRNN, self).__init__()
        self.conv_x = Conv()
        self.conv_y = Conv()
        self.conv_z = Conv()
        self.rnn = nn.RNN(input_size=2016, hidden_size=64)
        self.fla = nn.Flatten()
        self.fc1 = nn.Linear(64, 100)
        self.fc2 = nn.Linear(100, 2)

    def forward(self, data):
        x, y, z = data[:, 0, :], data[:, 1, :], data[:, 2, :]
        x = x.view(-1, 1, x.shape[-1])
        y = y.view(-1, 1, y.shape[-1])
        z = z.view(-1, 1, z.shape[-1])
        x_out = self.conv_x(x)
        y_out = self.conv_y(y)
        z_out = self.conv_z(z)
        new_feature = torch.cat([x_out, y_out, z_out], dim=1)
        new_feature = new_feature.view(-1, 1, new_feature.shape[-1])
        out, _ = self.rnn(new_feature)
        out = self.fla(out)
        out = F.relu(self.fc1(out))
        out = self.fc2(out)
        return F.log_softmax(out, dim=1)

In [10]:
data=torch.rand(16,3,400)
model=MCRNN()
model(data).shape

torch.Size([16, 2])

In [11]:
%%time
setup_seed(42)
model = MCRNN().cuda()
L = []
test_hist = []
optimizer = optim.Adam(model.parameters(), lr=0.001)
for epoch in range(1, 100 + 1):
    start = time.time()
    loss = train(model, train_loader, optimizer, epoch)
    L.append(loss)
    print("time {:.1f} sec:".format(time.time() - start))
    acc = test(model, test_loader)
    test_hist.append(acc)

Train Epoch: 1, Average Loss: 0.3891
time 1.2 sec:
Test set: Average loss: 0.3539, Accuracy: 1837/2069 (88.79%)

Train Epoch: 2, Average Loss: 0.3488
time 1.2 sec:
Test set: Average loss: 0.3509, Accuracy: 1837/2069 (88.79%)

Train Epoch: 3, Average Loss: 0.3481
time 1.2 sec:
Test set: Average loss: 0.3541, Accuracy: 1837/2069 (88.79%)

Train Epoch: 4, Average Loss: 0.3515
time 1.1 sec:
Test set: Average loss: 0.3511, Accuracy: 1837/2069 (88.79%)

Train Epoch: 5, Average Loss: 0.3498
time 1.1 sec:
Test set: Average loss: 0.3511, Accuracy: 1837/2069 (88.79%)

Train Epoch: 6, Average Loss: 0.3504
time 1.2 sec:
Test set: Average loss: 0.3510, Accuracy: 1837/2069 (88.79%)

Train Epoch: 7, Average Loss: 0.3490
time 1.1 sec:
Test set: Average loss: 0.3520, Accuracy: 1837/2069 (88.79%)

Train Epoch: 8, Average Loss: 0.3518
time 1.1 sec:
Test set: Average loss: 0.3512, Accuracy: 1837/2069 (88.79%)

Train Epoch: 9, Average Loss: 0.3493
time 1.1 sec:
Test set: Average loss: 0.3514, Accuracy: 183

Train Epoch: 73, Average Loss: 0.0123
time 1.1 sec:
Test set: Average loss: 0.0126, Accuracy: 2058/2069 (99.47%)

Train Epoch: 74, Average Loss: 0.0111
time 1.2 sec:
Test set: Average loss: 0.0142, Accuracy: 2057/2069 (99.42%)

Train Epoch: 75, Average Loss: 0.0097
time 1.1 sec:
Test set: Average loss: 0.0215, Accuracy: 2051/2069 (99.13%)

Train Epoch: 76, Average Loss: 0.0141
time 1.1 sec:
Test set: Average loss: 0.0226, Accuracy: 2052/2069 (99.18%)

Train Epoch: 77, Average Loss: 0.0122
time 1.1 sec:
Test set: Average loss: 0.0101, Accuracy: 2063/2069 (99.71%)

Train Epoch: 78, Average Loss: 0.0103
time 1.2 sec:
Test set: Average loss: 0.0110, Accuracy: 2059/2069 (99.52%)

Train Epoch: 79, Average Loss: 0.0099
time 1.2 sec:
Test set: Average loss: 0.0099, Accuracy: 2062/2069 (99.66%)

Train Epoch: 80, Average Loss: 0.0095
time 1.2 sec:
Test set: Average loss: 0.0120, Accuracy: 2058/2069 (99.47%)

Train Epoch: 81, Average Loss: 0.0100
time 1.1 sec:
Test set: Average loss: 0.0112, Accu

In [12]:
model.eval()
y_pred = []
y_test = []
with torch.no_grad():
    for data, target in test_loader:
        data = data.cuda()
        output = model(data)
        pred = output.max(1, keepdim=True)[1]  
        pred = pred.cpu().numpy().squeeze()
        y_pred.append(pred)
        y_test.append(target.numpy().squeeze())
y_pred=np.hstack(y_pred)
y_test=np.hstack(y_test)

summary(y_pred,y_test)

confusion matrix:
 [[1835    9]
 [   2  223]]
Roc score: 0.9931
F1 score: 0.9759
Recall_score: 0.9911
Precision_score: 0.9612
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      1844
           1       0.96      0.99      0.98       225

    accuracy                           0.99      2069
   macro avg       0.98      0.99      0.99      2069
weighted avg       0.99      0.99      0.99      2069



In [13]:
torch.save(model, './models/MCRNN.pth')